# Imports

In [ ]:
# Import packages

import warnings
warnings.filterwarnings('ignore',category = FutureWarning)

# Data
import numpy  as np
import pandas as pd; pd.set_option('display.max_columns', None)

# Graphs
import matplotlib.pyplot as plt; plt.style.use('seaborn-darkgrid')
import seaborn as sns; sns.set()

# Others
import math; import random as rd; import sys; import os; import random
from tqdm.notebook import tqdm
from tqdm.keras import TqdmCallback
import json

# Tensorflow
import tensorflow as tf

# SKLearn
import sklearn
import sklearn.model_selection
from sklearn import impute

# Fix seeds
seed = 0
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

# Get current time to save files
from datetime import datetime

In [ ]:
# Loading TensorFlow variables and defining functions
print(tf.__version__) # Must be >2 for Tensorflow Probability

Input              = tf.keras.layers.Input
Dense              = tf.keras.layers.Dense
BatchNormalization = tf.keras.layers.BatchNormalization
Dropout            = tf.keras.layers.Dropout
Model              = tf.keras.models.Model
Nadam              = tf.keras.optimizers.Nadam

# Loading, pre-processing and scaling data

In [ ]:
from sklearn.preprocessing import StandardScaler

def MakeSamples(All_Data, Test_Size=0.30, Bins=None, Plot_Hists=False, Seed=None, FixAfter=False, DisplayDFs=False, Show_Scaled=False):
    print('##################################################################')
    print('# Generating new samples')
    print('##################################################################')
    if Seed == None:
        Seed = np.random.randint(0, 2**30)
        np.random.seed(Seed)
    else:
        np.random.seed(Seed)
    print('# Random seed             = %s' %Seed)
    
    TrainingSample, TestingSample = sklearn.model_selection.train_test_split(All_Data, test_size=Test_Size, random_state=Seed)
    
    print('# First 5 train. samples  = %s' %TrainingSample.index.values[:5])
    print('# First 5 testing samples = %s' %TestingSample.index.values[:5])
  
    ##############################################################
    # Verifying the number of objects in each sample
    print('# Training Sample Objects = %s, Percentage of total = %.3g%%' %(len(TrainingSample), (100*len(TrainingSample)/(len(TrainingSample)+len(TestingSample)))))
    print('# Testing Sample Objects  = %s, Percentage of total = %.3g%%' %(len(TestingSample), (100*len(TestingSample)/(len(TrainingSample)+len(TestingSample)))))
    print('# Total Sample Objects    = %s' %(len(TestingSample)+len(TrainingSample)))
    # Verifying the existence of duplicates in the samples
    print('# Number of matching rows between two samples (should be zero):')
    print('# Train/Test = %s' %len(pd.merge(pd.DataFrame(TrainingSample), pd.DataFrame(TestingSample), how='inner')))
    # Statistics
    print('# Train max Z = %s' %np.max(TrainingSample.z_SDSS))
    print('# Test max Z  = %s' %np.max(TestingSample.z_SDSS))
  
    ##############################################################
    # Display DFs
    if DisplayDFs == True:
        display(Train_Data[:3])
        display(Test_Data[:3])
  
    ##############################################################
    # Scaling Data
    Scaler = StandardScaler()
  
    Scaled_Train_X = Scaler.fit_transform(TrainingSample[TrainingFeatures])
    Scaled_Train_X = pd.DataFrame(Scaled_Train_X, columns=[TrainingFeatures])
  
    Scaled_Test_X = Scaler.transform(TestingSample[TrainingFeatures])
    Scaled_Test_X = pd.DataFrame(Scaled_Test_X, columns=[TrainingFeatures])
  
    # Fix missing features after scaling
    if FixAfter == True:
        for feature in Features:
            Scaled_Train_X.loc[TrainingSample.reset_index(drop=True)[feature] == 0, feature] = 0
            Scaled_Test_X .loc[TestingSample.reset_index(drop=True)[feature] == 0, feature] = 0
  
    ##############################################################
    # Plot Feature Histograms
    if Plot_Hists == True:
        fig, ax = plt.subplots(figsize=(15,15))
        plt.subplots_adjust(hspace=0.5, wspace=0.5)
        if Show_Scaled == False:
            Features_to_plot = Features + Extra_F + Colors + zBPZ + Target
        if Show_Scaled == True:
            Features_to_plot = Features + Extra_F + Colors
        print()
        plt_idx = 1
        for feature in Features_to_plot:
            plt.subplot(7, 5, plt_idx)
    
            if Show_Scaled == False:
                Feature_min = min(np.min(TrainingSample[feature]), np.min(TestingSample[feature]))
                Feature_max = max(np.max(TrainingSample[feature]), np.max(TestingSample[feature]))
      
                plt.hist(TrainingSample[feature], lw=2, range=(Feature_min, Feature_max), bins=20, histtype='step')
                plt.hist(TestingSample[feature], lw=2, range=(Feature_min, Feature_max), bins=20, histtype='step')
    
            if Show_Scaled == True:
                Feature_min = min(np.min(Scaled_Train_X[[feature]].values), np.min(Scaled_Train_X[[feature]].values))
                Feature_max = max(np.max(Scaled_Train_X[[feature]].values), np.max(Scaled_Train_X[[feature]].values))
      
                plt.hist(Scaled_Train_X[[feature]].values, lw=2, range=(Feature_min, Feature_max), bins=20, histtype='step')
                plt.hist(Scaled_Test_X[[feature]].values, lw=2, range=(Feature_min, Feature_max), bins=20, histtype='step')
    
            plt.yscale('log')
    
            plt.xlabel(feature)
    
            plt.grid(lw=.5)
            plt_idx = plt_idx+1  
    
        fig.tight_layout()
        plt.show()
      
    return TrainingSample, Scaled_Train_X, TestingSample, Scaled_Test_X

In [ ]:
# Loading and pre-processing input data
# Fix seed
np.random.seed(seed)

# Loading CSVs
All_Data = pd.read_csv('AllData.csv')

# Preprocessing Data
C0       = All_Data['r_auto']        .between(16,21)
C1       = All_Data['PhotoFlag']     == 0
C2       = All_Data['PROB_GAL']      >= 0.5
C3       = All_Data['z_SDSS']        >= 1e-4
C4       = All_Data['zErr']          <= 0.4 
C5       = All_Data['class_SDSS']    != 'QSO'
C6       = All_Data['nDet_auto']     >= 8
All_Data = All_Data[C0 & C1 & C2 & C3 & C4 & C5 & C6]

# Calculating ellipticity (flattening)
All_Data['Ellipticity'] = 1 - All_Data['B']/All_Data['A']

# Defining column names
Extra_F   = ['FWHM_n', 'MUMAX', 'Ellipticity']
Features  = [s for s in All_Data.columns.values if ('auto' in s or 'mag' in s) and not (s.startswith('e') or s.startswith('n') or s.startswith('s') or s.endswith('err'))]
Errors    = [s for s in All_Data.columns.values if (('auto' in s) or ('_mag' in s)) and (s.startswith('e') or s.endswith('err'))]
Target    = ['z_SDSS']
Target_er = ['zErr']
zBPZ      = ['zb']

# Replace missing magnitudes by zero
for feature in Features:
  All_Data.loc[All_Data[feature] < 0,   feature] = 0
  All_Data.loc[All_Data[feature] > 50,  feature] = 0

# Calculate colours
Reference_Band  = 'r_auto'
Reference_Idx   = Features.index(Reference_Band)
FeaturesToLeft  = Features[:Reference_Idx]
FeaturesToRight = Features[(Reference_Idx+1):]

for feature in FeaturesToLeft: # of Reference_Band
  All_Data[feature+'-'+Reference_Band] = All_Data[feature] - All_Data[Reference_Band] 
for feature in FeaturesToRight:
  All_Data[Reference_Band+'-'+feature] = All_Data[Reference_Band] - All_Data[feature]

Colours = [s for s in All_Data.columns.values if ('-' in s and 'auto' in s)]

# Fix colors from missing features
for colour in Colours:
    All_Data.loc[All_Data[colour] <= -10,  colour] = -10
    All_Data.loc[All_Data[colour] >= 10,   colour] = 10

TrainingFeatures =  Features + Colours + Extra_F

print('# Features:\n# %s' %(TrainingFeatures))
print('# Target:\n# %s' %Target)
print('# Errors:\n# %s' %Errors)

In [ ]:
TrainingSample, Training_Data_Features, TestingSample, Testing_Data_Features = MakeSamples(All_Data, Test_Size=0.30, Bins=200, Plot_Hists=False, Seed=seed, FixAfter=True, Show_Scaled=True)

In [ ]:
# Defining features
Training_Data_Features = Training_Data_Features[TrainingFeatures].values
Testing_Data_Features  = Testing_Data_Features[TrainingFeatures].values

# Defining target
Training_Data_Target = TrainingSample['z_SDSS'].values
Testing_Data_Target = TestingSample['z_SDSS'].values

## Training

In [ ]:
# TFP
import tensorflow_probability as tfp
tfd = tfp.distributions

# Negative Log Likelihood loss
def neglogik(y, p_y):
    return -p_y.log_prob(y)

DenseVariational = tfp.layers.DenseVariational

# Posterior definition
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2 * n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(tfd.Normal(loc=t[..., :n], scale=1e-5 + tf.nn.softplus(t[..., n:])), reinterpreted_batch_ndims=1)),
    ])

# Prior definition
def prior_trainable(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(tfd.Normal(loc=t, scale=.1), reinterpreted_batch_ndims=1)),
    ])

In [ ]:
# Fix seeds. Seed is defined in the import cell
random.seed(seed)
np.random.seed(seed)
tf.compat.v1.random.set_random_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

lrelu      = tf.keras.layers.LeakyReLU()
Units      = 196
Epochs     = 500
Num_Layers = 3
Batch_Size = 256

# MDN Output config
num_components = 20 # Number of Gaussians in the output mixture
event_shape = [1]   # Dimension of the output (1 because we want the photometric redshift)
params_size = tfp.layers.MixtureNormal.params_size(num_components, event_shape) # Number of units needed in the layer before the MixtudeSameFamily layer

lr        = 0.001
clipvalue = 0.5
clipnorm  = 0.5

# Dense Variational model
tf.keras.backend.clear_session()
def Dense_Variational(TrainSampleSize):
    Input_Layer = Input(shape=(np.shape(Training_Data_Features)[1],))
    
    for i in range(Num_Layers):
        if i == 0:
            l = DenseVariational(Units, posterior_mean_field, prior_trainable, kl_weight=1/TrainSampleSize, activation=lrelu)(Input_Layer)
        else:
            l = DenseVariational(Units, posterior_mean_field, prior_trainable, kl_weight=1/TrainSampleSize, activation=lrelu)(l)
        l = BatchNormalization()(l)
        l = Dropout(0.1)(l)

    l = Dense(params_size)(l)
    
    out = tfp.layers.MixtureNormal(num_components, event_shape)(l)
    
    model = Model(inputs=Input_Layer, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Nadam(lr=lr, clipvalue=clipvalue, clipnorm=clipnorm), loss=neglogik)

    return model

In [ ]:
# Train model
# Fix seed
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

# Current time and output folder
TimeNow = datetime.now().strftime("%d-%m-%Y/%Hh%Mm/")
Output_Dir = 'Results/MDN/'+ TimeNow
print("# Output_Dir = '%s'" %Output_Dir)
if os.path.isdir(Output_Dir) == False:
    os.makedirs(Output_Dir)
        
# K-Fold?
Scheme = 'KFold'

if Scheme == 'KFold':
    i = 0
    print('# K-Fold seed: %s' %seed)
    kfold = sklearn.model_selection.KFold(n_splits=4, shuffle=True, random_state=seed)

    # Save each fit separately
    Prob_Model     = {}
    Prob_Model_Fit = {}
    for train, validation in kfold.split(Training_Data_Features, Training_Data_Target):
        print('##################################################################')
        print('# Starting fold number %s' %i)
        print('# Training start time: %s' %TimeNow)
        print('# Output directory is  %s' %Output_Dir)
        print('##################################################################')
        print('# Training Sample Objects   = %s (%.3g%%)' %(len(train), (100*len(train)/(len(train)+len(validation)+len(Testing_Data_Features)))))
        print('# Validation Sample Objects = %s (%.3g%%)' %(len(validation), (100*len(validation)/(len(train)+len(validation)+len(Testing_Data_Features)))))
        print('# Testing Sample Objects    = %s (%.3g%%)' %(len(Testing_Data_Features), (100*len(Testing_Data_Features)/(len(train)+len(validation)+len(Testing_Data_Features)))))
        print('#')
        print('# Number of matching rows between two samples (should be zero):')
        print('# Train/Validation = %s' % len(pd.merge(pd.DataFrame(Training_Data_Features[train]), pd.DataFrame(Training_Data_Features[validation]), how='inner')))
        print('# Test/Validation  = %s' % len(pd.merge(pd.DataFrame(Testing_Data_Features), pd.DataFrame(Training_Data_Features[validation]), how='inner')))
        print('# Train/Test       = %s' % len(pd.merge(pd.DataFrame(Training_Data_Features[train]), pd.DataFrame(Testing_Data_Features), how='inner')))
        print()

        # Custom metrics
        warnings.filterwarnings('ignore', category = Warning)

        # Compiling new model for each fold
        Prob_Model[i] = Dense_Variational(len(Training_Data_Features))

        # Fitting the model
        Prob_Model_Fit[i] = Prob_Model[i].fit(Training_Data_Features[train], Training_Data_Target[train],
                                              validation_data=(Training_Data_Features[validation], Training_Data_Target[validation]), 
                                              epochs=Epochs, batch_size=Batch_Size, verbose=0, callbacks=[TqdmCallback(verbose=0)])

        i = i+1
        print()

In [ ]:
# Plot and save Loss
Folds = Prob_Model_Fit.keys()

fig, ax = plt.subplots(figsize=(14, 7))
plt_idx = 1
for fold in Folds:
    plt.subplot(2, 2, plt_idx)
    
    plt.plot(Prob_Model_Fit[fold].history['loss'], lw=2, alpha=1, label='Training')
    plt.plot(Prob_Model_Fit[fold].history['val_loss'], lw=2, alpha=1, label='Validation')
    plt.ylim(top=-1, bottom=-2)
    
    plt.ylabel('Loss (NGL)')
    plt.xlabel('Epochs')
    if plt_idx == 1:
        plt.legend()

    plt_idx = plt_idx+1
    
fig.tight_layout()
plt.savefig(Output_Dir+'Loss.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Save model
Save_Model = True
Folds = Prob_Model_Fit.keys()

if Save_Model == True:
    for fold in Folds:
        # Save model in TF format
        Prob_Model[fold].save(Output_Dir+'Fold_%s' %fold, overwrite=True)

        # Save training history
        pd.DataFrame(Prob_Model_Fit[fold].history).to_csv(Output_Dir+'/Seed'+str(seed)+'_Fold%s.csv' %fold, index=False)

In [ ]:
# Make predictions
from scipy import integrate # To calculate the CDF of objects

# To calculate PDFs
def Calc_PDF(x, Weights, Means, STDs):
    PDF = np.sum(Weights*(1/(STDs*np.sqrt(2*np.pi))) * np.exp((-1/2) * ((x[:,None]-Means)**2)/(STDs)**2), axis=1)
    return PDF/np.trapz(PDF, x)

# General function to find the nearest idx of an item in a list
def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

# Step function
def step(x,y):
    return 1 * (x > y)

# Calculate HPDCI per object
# https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def Check_Intervals(x):
    List, last = [[]], None
    for elem in x:
        if last is None or abs(last - elem) <= 1:
            List[-1].append(elem)
        else:
            List.append([elem])
        last = elem
    return List

def Calculate_HPDCI(x, pdf_object, zspec):
    HPDCI_Indexes = list(np.where(pdf_object >= pdf_object[find_nearest_idx(x, zspec)])[0])
    HPDCI_Indexes = Check_Intervals(HPDCI_Indexes)

    Object_HPDCI = 0
    for k in range(len(HPDCI_Indexes)):
        Object_HPDCI += np.trapz(pdf_object[HPDCI_Indexes[k]], x[HPDCI_Indexes[k]])

    return Object_HPDCI

Folds = Prob_Model.keys()

if len(Folds) > 1:
    print('# Predicting for %s folds' %len(Folds))

    x = np.arange(0, 1+0.001, 0.001)
    MC_each_fold = 500

    Fold_Weights = {}
    Fold_Means   = {}
    Fold_STDs    = {}
    Fold_PDFs    = {}
    Fold_PhotoZs = {}
    Fold_CRPS    = {}
    Fold_PITs    = {}
    Fold_Odds    = {}

    for fold in tqdm(Folds):
        MC_Weights = []
        MC_Means   = []
        MC_STDs    = []

        for i in tqdm(range(MC_each_fold)):
            Pred = Prob_Model[fold](Testing_Data_Features)

            Weight = Pred.submodules[1].probs_parameter().numpy()                                                 # Weights
            Mean   = Pred.submodules[0].mean().numpy().reshape(len(Testing_Data_Features), np.shape(Weight)[1])   # Means
            Std    = Pred.submodules[0].stddev().numpy().reshape(len(Testing_Data_Features), np.shape(Weight)[1]) # Stds

            Sorted_Weight_Index = np.flip(np.argsort(Weight, axis=1), axis=1)

            MC_Weights.append(Weight[np.arange(len(Weight))[:,None], Sorted_Weight_Index]) # Appending them to a list, so we can take the median below
            MC_Means.append(Mean[np.arange(len(Mean))[:,None], Sorted_Weight_Index])       # https://stackoverflow.com/questions/20103779/index-2d-numpy-array-by-a-2d-array-of-indices-without-loops
            MC_STDs.append(Std[np.arange(len(Std))[:,None], Sorted_Weight_Index])          #

        Fold_Weights[fold] = np.median(MC_Weights, axis=0)
        Fold_Means  [fold] = np.median(MC_Means, axis=0)
        Fold_STDs   [fold] = np.median(MC_STDs, axis=0)

        # Calculate ZPhots, PITs, CRPS, and Odds per fold
        Fold_PhotoZs[fold] = []
        Fold_Odds[fold]    = []
        Fold_PITs[fold]    = []
        Fold_CRPS[fold]    = []

        for i in range(len(Testing_Data_Features)):
            # Get the PDF for the object i
            Obj_PDF      = Calc_PDF(x, Fold_Weights[fold][i], Fold_Means[fold][i], Fold_STDs[fold][i])
            # Find an approximate photo-z
            Approx_Zphot = x[np.argmax(Obj_PDF)]
            # Using the approx photo-z, define a new x-grid and calculate the PDF using it (this makes a 'detailed' PDF peak)
            x_detailed   = np.linspace(Approx_Zphot-0.025, Approx_Zphot+0.025, 101)
            # Using the new x-grid, build the 'detailed' PDF and find a better photo-z
            Fold_PhotoZs[fold].append(x_detailed[np.argmax(Calc_PDF(x_detailed, Fold_Weights[fold][i], Fold_Means[fold][i], Fold_STDs[fold][i]))])

            # From the Obj_PDF, calculate the CDF
            Obj_CDF = integrate.cumtrapz(Obj_PDF, x, initial=0)
            # Calculate the Odds of object i (arXiv 9811189, eq. 17. Also calculated as the integral of the PDF between z_peak +/- 0.02)
            Fold_Odds[fold].append( Obj_CDF[find_nearest_idx(x, Fold_PhotoZs[fold][i]+0.02)] - Obj_CDF[find_nearest_idx(x, Fold_PhotoZs[fold][i]-0.02)] )
            # Calculate the PIT of object i (arXiv 1608.08016, eq. 2)
            Fold_PITs[fold].append( Obj_CDF[find_nearest_idx(x, TestingSample['z_SDSS'].values[i])] )
            # Calculate the CRPS of object i (arXiv 1608.08016, eq. 4)
            Fold_CRPS[fold].append( np.trapz((Obj_CDF - step(x, TestingSample['z_SDSS'].values[i]))**2, x) )

    Final_Weights = np.median([Fold_Weights[fold] for fold in Folds], axis=0)
    Final_Means   = np.median([Fold_Means[fold] for fold in Folds], axis=0)
    Final_STDs    = np.median([Fold_STDs[fold] for fold in Folds], axis=0)

    # PDF
    PDF = tfd.MixtureSameFamily(mixture_distribution    = tfd.Categorical(probs=Final_Weights),
                                components_distribution = tfd.Normal(loc=Final_Means, scale=Final_STDs))
    PDF_STDs = PDF.stddev().numpy()

    Final_PDFs = []
    for i in range(len(Testing_Data_Features)):
        Final_PDFs.append(Calc_PDF(x, Final_Weights[i], Final_Means[i], Final_STDs[i]))

    # Calculate ZPhots, PITs, CRPS, and Odds per fold
    Fine_ZPhot = []
    Odds       = []
    PITs       = []
    CRPS       = []

    for i in range(len(Testing_Data_Features)):
        # Get the PDF for the object i
        Obj_PDF      = Calc_PDF(x, Final_Weights[i], Final_Means[i], Final_STDs[i])
        # Find an approximate photo-z
        Approx_Zphot = x[np.argmax(Obj_PDF)]
        # Using the approx photo-z, define a new x-grid and calculate the PDF using it (this makes a 'detailed' PDF peak)
        x_detailed   = np.linspace(Approx_Zphot-0.025, Approx_Zphot+0.025, 101)
        # Using the new x-grid, build the 'detailed' PDF and find a better photo-z
        Fine_ZPhot.append(x_detailed[np.argmax(Calc_PDF(x_detailed, Final_Weights[i], Final_Means[i], Final_STDs[i]))])

        # From the Obj_PDF, calculate the CDF
        Obj_CDF = integrate.cumtrapz(Obj_PDF, x, initial=0)
        # Calculate the Odds of object i (arXiv 9811189, eq. 17. Also calculated as the integral of the PDF between z_peak +/- 0.02)
        Odds.append( Obj_CDF[find_nearest_idx(x, Fine_ZPhot[i]+0.02)] - Obj_CDF[find_nearest_idx(x, Fine_ZPhot[i]-0.02)] )
        # Calculate the PIT of object i (arXiv 1608.08016, eq. 2)
        PITs.append( Obj_CDF[find_nearest_idx(x, TestingSample['z_SDSS'].values[i])] )
        # Calculate the CRPS of object i (arXiv 1608.08016, eq. 4)
        CRPS.append( np.trapz((Obj_CDF - step(x, TestingSample['z_SDSS'].values[i]))**2, x) )

    Result_DF = pd.DataFrame()
    Result_DF['r_auto']      = TestingSample['r_auto'].values
    Result_DF['z']           = TestingSample['z_SDSS'].values
    Result_DF['class_SDSS']  = TestingSample['class_SDSS'].values
    Result_DF['zml']         = Fine_ZPhot
    Result_DF['Odds']        = Odds
    Result_DF['PIT']         = PITs
    Result_DF['CRPS']        = CRPS